## Import libraries

In [1]:
import cv2
import mediapipe as mp
from deepface import DeepFace

### Initialize Mediapipe

In [2]:
# Initialize MediaPipe Face Detection and Drawing
mp_face_detection = mp.solutions.face_detection
mp_draw = mp.solutions.drawing_utils

### Initialize video capture

In [4]:
# Initialize video capture
cap = cv2.VideoCapture(0)


### Intializes face detection and emotion analysis ,and stops the video capture



In [5]:
with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty frame...")
            continue

        # Convert the frame to RGB as MediaPipe processes RGB images
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform face detection
        results = face_detection.process(rgb_frame)

        # Draw detection results on the frame and perform emotion detection
        if results.detections:
            for detection in results.detections:
                # Draw bounding box around the face
                mp_draw.draw_detection(frame, detection)

                # Get bounding box coordinates
                bbox_c = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape
                bbox = int(bbox_c.xmin * w), int(bbox_c.ymin * h), \
                       int(bbox_c.width * w), int(bbox_c.height * h)

                # Extract the face from the frame
                face = frame[bbox[1]:bbox[1] + bbox[3], bbox[0]:bbox[0] + bbox[2]]

                # Perform emotion detection
                try:
                    analysis = DeepFace.analyze(face, actions=['emotion'], enforce_detection=False)
                    
                    # Handle analysis output structure
                    if isinstance(analysis, dict):
                        emotion = analysis.get('dominant_emotion', 'Unknown')
                    elif isinstance(analysis, list) and len(analysis) > 0:
                        emotion = analysis[0].get('dominant_emotion', 'Unknown')
                    else:
                        emotion = 'Unknown'
                    
                    # Display detected emotion on the frame
                    cv2.putText(frame, f'Emotion: {emotion}', (bbox[0], bbox[1] - 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                except Exception as e:
                    print(f"Error in emotion detection: {e}")

                # Displaying the confidence score on the frame
                score = detection.score[0]
                cv2.putText(frame, f'Score: {int(score * 100)}%', (bbox[0], bbox[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Display the output frame
        cv2.imshow('Face Detection with Emotion', frame)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()